In [1]:
import face_recognition
import cv2
import numpy as np
import DataStore

In [ ]:
for j in db:
    print(db[j])

In [2]:
video_capture = cv2.VideoCapture(0)

In [3]:
db = DataStore.DB
known_face_encodings = []
known_face_names =[]
n = len(db)

for j in db:
    i = str(db[j]['ID'])
    
    known_face_names.append(db[j]['Name'])
    
    ImCode = 'image' + i + ' = face_recognition.load_image_file("' + db[j]['FileName'] + '")'
    exec(ImCode)
    
    EnCode = 'face_encoding_' + i + '= face_recognition.face_encodings(image' + i + ')[0]'
    exec(EnCode)
    
    ApCode = 'known_face_encodings.append(face_encoding_' + i + ')'
    exec(ApCode)

In [4]:
from csv import writer
from csv import DictWriter

field_names = ['ID', 'Name', 'Age', 'Gender', 'Number', 'Balance', 'FileName']
file_name = 'PeopleSpotted.csv'

with open(file_name, 'a+', newline='') as write_obj:
    csv_writer = writer(write_obj)
    csv_writer.writerow(field_names)

    
def HandleFace(name):
    for j in db:
        if name == db[j]['Name']:
        
            with open(file_name, 'a+', newline='') as write_obj:
                dict_writer = DictWriter(write_obj, fieldnames=field_names)
                dict_writer.writerow(db[j])

In [5]:
checklist = [False]*n
BigCheck = True
def ExtractInfo(mathes):
    if not False in checklist: BigCheck = False
        
    if not True in matches: print('Unknown')
    else:
        for i in range(n):
            if (checklist[i] == False) and (matches[i] == True):
                HandleFace(known_face_names[i])
                checklist[i] = True
    

In [6]:
# Initialize these variables to pre allot space
face_encodings, face_locations, face_names, process_this_frame = [], [], [], True

# Run loop to capture and give output
while True:
    
    ret, frame = video_capture.read()
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25) #Resizing captured frame
    rgb_small_frame = small_frame[:, :, ::-1] #BGR -> RGB

    if process_this_frame: # Processing only half the frames for speed
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            if BigCheck: ExtractInfo(matches)
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
            face_names.append(name)

    process_this_frame = not process_this_frame


    for (top, right, bottom, left), name in zip(face_locations, face_names): #Rescale
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        
        #Style the Box
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    cv2.imshow('Video', frame) #Display output box in video

    if cv2.waitKey(1) & 0xFF == ord('q'): # Quit if Q/q is pressed
        break

video_capture.release()
cv2.destroyAllWindows()

Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
